<a href="https://colab.research.google.com/github/yilenpan/quantized_lora/blob/main/Udacity_PEFT_training_GPT2_for_sequence_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the necessary packages

In [2]:
!pip install transformers "datasets==2.15.0" torch peft numpy bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00


Loading DAIR-AI/emotion

In [3]:
from datasets import load_dataset


# The dair-ai/emotion dataset has three splits
splits = ["train", "test", "validation"]

data_splits = load_dataset("dair-ai/emotion")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Split and format data. The inputs for the model have to have certain keywords removed. Only labels, input_ids, and attention_mask can be used. You must also make sure the inputs are pytorch tensors, and flat.

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(x):
    return tokenizer(x['text'], truncation=True, padding=True)


def format_dataset(ds):
    ds = ds.remove_columns('text')
    ds = ds.rename_column('label', 'labels')
    ds.set_format('torch', columns=['labels', 'input_ids', 'attention_mask'])
    return ds

test_split = data_splits["test"].map(preprocess_function)
train_split = data_splits["train"].map(preprocess_function)
val_split = data_splits["validation"].map(preprocess_function)

test_split = format_dataset(test_split)
train_split = format_dataset(train_split)
val_split = format_dataset(val_split)


print(test_split)
print(val_split)

print(train_split)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 2000
})
Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 2000
})
Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 16000
})


Let's quickly evaluate the performance of the pretrained model. We're going to load gpt2, attach an additional layer with 6 outputs, then see what we get.

In [30]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, AutoModelForSequenceClassification
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

label2id={'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}
id2label={0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}
model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=6,
    label2id=label2id,
    id2label=id2label,
    pad_token_id=tokenizer.eos_token_id,
)
trainer = Trainer(
    model=model,
    train_dataset=train_split,
    eval_dataset=test_split,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,

)

results = trainer.evaluate()
print(f"Accuracy: {results['eval_accuracy'] * 100}%")


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy: 11.200000000000001%


Ooooof. we're looking at around 10% out of the base model. Let's see if we can do better.

As a test run, let's see what we get if we pass input into the model

In [28]:
def get_label(output):
    logits = outputs.logits
    logits = logits.float()
    probabilities = torch.softmax(logits, dim=-1)
    return torch.argmax(probabilities, dim=-1).item()

# Testing out the model no training
test_row = test_split[0]
outputs = model(test_row['input_ids'].view(1, -1).to('cuda'))
predicted_label = get_label(outputs)
print(f"(predicted label: {predicted_label}, actual label: {test_row['labels']})")


(predicted label: 4, actual label: 0)


Use Bytes and Bytes to quantize the model. This converts the data type for the weights from a 32bit number to a 4 bit number. Then load the model with the quantization config. This runs the model at lower precision.

Because we are doing a classification task, use the AutoModelForSeqenceClassification package. Add a layer to the model that only has 6 outputs. Finally, freeze the model. We only want to train the lora.

In [41]:

from peft import prepare_model_for_kbit_training
from transformers import AutoModelForSequenceClassification, BitsAndBytesConfig
import torch

q_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)


model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=6,
    label2id=label2id,
    id2label=id2label,
    pad_token_id=tokenizer.eos_token_id,
    quantization_config=q_config,
)

model = prepare_model_for_kbit_training(model)


# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

model


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear4bit(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear4bit(in_features=768, out_features=768, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Linear4bit(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear4bit(in_features=3072, out_features=768, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, ele

Create a Lora config, and attach it to the layers in then Attention mechanism. Loras are matricies that are multiplied by the weights in these layers.

In [33]:
from peft import LoraConfig, get_peft_model, TaskType

config = LoraConfig(
    r=16, # Rank
    lora_alpha=8,
    bias="lora_only",
    target_modules=['c_attn', 'c_proj'], #Assign correct layers (see model)
    lora_dropout=0.01,
    task_type=TaskType.SEQ_CLS, # WTF NOT IN THE DOCS!!!!!!
    fan_in_fan_out=True
)

lora_model = get_peft_model(model, config)

print(lora_model.print_trainable_parameters())

trainable params: 1,626,624 || all params: 126,071,040 || trainable%: 1.2902439767293108
None


Create the trainer class

In [34]:

training_args = TrainingArguments(
    output_dir="./data/gpt2-lora",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    num_train_epochs=15,
    per_device_train_batch_size=50, # Kick this up if we can
    per_device_eval_batch_size=50, # Kick this up if we can
    learning_rate=2e-5,
    save_strategy="epoch",
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_split,
    eval_dataset=test_split,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

print(test_split)
print(test_split[0])

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 2000
})
{'labels': tensor(0), 'input_ids': tensor([  320,  4203,  2138, 36371,   523,   545,   407,   845, 14742,   826,
          783]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}


Let's Train!

In [36]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.630301,0.350000
2,2.912000,1.523499,0.433000
3,2.912000,1.336838,0.535000
4,1.468300,1.044790,0.617000
5,1.109100,0.792806,0.714000
6,1.109100,0.655055,0.762000
7,0.822900,0.582895,0.792000
8,0.706500,0.543913,0.805500
9,0.706500,0.514165,0.814000
10,0.640300,0.496394,0.817000


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.630301,0.350000
2,2.912000,1.523499,0.433000
3,2.912000,1.336838,0.535000
4,1.468300,1.044790,0.617000
5,1.109100,0.792806,0.714000
6,1.109100,0.655055,0.762000
7,0.822900,0.582895,0.792000
8,0.706500,0.543913,0.805500
9,0.706500,0.514165,0.814000
10,0.640300,0.496394,0.817000


TrainOutput(global_step=4800, training_loss=1.0185364405314128, metrics={'train_runtime': 2083.6284, 'train_samples_per_second': 115.184, 'train_steps_per_second': 2.304, 'total_flos': 6575920045056000.0, 'train_loss': 1.0185364405314128, 'epoch': 15.0})

Save the LoRa for later use!

In [37]:
lora_model.save_pretrained("mygpt2lora")

evaluate the lora - 82%!

In [42]:
from peft import AutoPeftModelForSequenceClassification

reloaded_model = AutoPeftModelForSequenceClassification.from_pretrained(
    "mygpt2lora",
    num_labels=6,
    label2id=label2id,
    id2label=id2label,
    pad_token_id=tokenizer.eos_token_id,
    quantization_config=q_config
)

trainer.model = reloaded_model
trainer.evaluate()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.487060546875,
 'eval_accuracy': 0.819,
 'eval_runtime': 9.2197,
 'eval_samples_per_second': 216.927,
 'eval_steps_per_second': 4.339,
 'epoch': 15.0}

In [43]:
outputs = reloaded_model(test_row['input_ids'].view(1, -1))
predicted_label = get_label(outputs)
print(f"(predicted label: {predicted_label}, actual label: {test_row['labels']})")

(predicted label: 0, actual label: 0)
